In [1]:
# extract information from dump file.
import numpy as num
import matplotlib.pyplot as plt
import pylab as lab
from scipy import constants
import scipy as sci
import pandas as pan
import os
%matplotlib inline

In [ ]:
def DumpExtractor(filename,frames,atomNumber,atomPlate):
    
    """
    infoDict=DumpExtractor(filename,frames,atomNumber,atomPlate)


    Function that extracts the information from the .dump file created by openmd
    
    
        Inputs:
      ===========
   
   
       filename:
   
               Path of the dump file from which the information is to be extracted
               
        frame:
    
                Total number of frames in the dump file
                
        atomNumber:
        
                Totla number of atoms in the slab or crystal
                
        atomPlate:
    
                Total number of atoms in the capacitor plates



        Outputs:
     =============
 
     infoDict:
 
         Dictonary containing position, velocity, chargeQV, electricField, plateEQV.
         Postion is a list of [x,y,z] and each x,y,z are array of x[frames][sites]
         velocity is a list of [vx,vy,vz] and each vx,vy,vz are array of vx[frames][sites]
         chargeQV is a lisf of [c,cv] and each c and cv are array of c[frame][sites]
         electric field is list of [ex,ey,ez] and each are array of ex[frame][sites]
         plateEQV is the list of [pex,pey,pez,pc,pcv] and each are array of pex[frames][sites]
    """
    
    fileDump=open(filename)  #dump file for info extraction
    linesDump=fileDump.readlines()

    processP="Wait"
    processC="Wait"


    #information storage matrix 
    #posiiton and velocity storage
    x=num.zeros((frames,atomNumber+1))
    y=num.zeros((frames,atomNumber+1))
    z=num.zeros((frames,atomNumber+1))
    vx=num.zeros((frames,atomNumber+1))
    vy=num.zeros((frames,atomNumber+1))
    vz=num.zeros((frames,atomNumber+1))


    #charge and velocity storage matrix
    c=num.zeros((frames,atomNumber+1))
    cv=num.zeros((frames,atomNumber+1))
    ex=num.zeros((frames,atomNumber+1))
    ey=num.zeros((frames,atomNumber+1))
    ez=num.zeros((frames,atomNumber+1))
    pc=num.zeros((frames,atomPlate))
    pcv=num.zeros((frames,atomPlate))
    pex=num.zeros((frames,atomPlate))
    pey=num.zeros((frames,atomPlate))
    pez=num.zeros((frames,atomPlate))

    #frame count initilization
    fCount=0
    index=0  #index for the atoms
    for line in linesDump:
        linesSplit=str.split(line)
        length=len(linesSplit)
    
        if(length!=0 and linesSplit[0]=="<StuntDoubles>" and processP=="Wait"):
            processP="Start"
            continue;
        
        elif(length!=0 and linesSplit[0]=="</StuntDoubles>" and processP=="Start"):
            processP="Wait"
            index=0
            continue;
        
        elif(length!=0 and linesSplit[0]=="<SiteData>" and processC=="Wait"):
            processC="Start"
            continue;
        
        elif(length!=0 and linesSplit[0]=="</SiteData>" and processC=="Start"):
            fCount=fCount+1
            index=0;
            processC="Wait"
            continue;
   
        elif(fCount>=frames):
            break;
        
        else:
            processP=processP;
            processC=processC;
        
        
        if (processP=="Start"):
            x[fCount][int(linesSplit[0])]=float(linesSplit[2])
            y[fCount][int(linesSplit[0])]=float(linesSplit[3])
            z[fCount][int(linesSplit[0])]=float(linesSplit[4])
            vx[fCount][int(linesSplit[0])]=float(linesSplit[5])
            vy[fCount][int(linesSplit[0])]=float(linesSplit[6])
            vz[fCount][int(linesSplit[0])]=float(linesSplit[7])
        
        if(processC=="Start"):
            if int(linesSplit[0])<atomNumber:
                c[fCount][int(linesSplit[0])]=float(linesSplit[3])
                cv[fCount][int(linesSplit[0])]=float(linesSplit[4])
                ex[fCount][int(linesSplit[0])]=float(linesSplit[5])
                ey[fCount][int(linesSplit[0])]=float(linesSplit[6])
                ez[fCount][int(linesSplit[0])]=float(linesSplit[7])
            elif (int(linesSplit[0])==atomNumber and linesSplit[1]=="cwe"):
                continue
                c[fCount][int(linesSplit[0])]=float(linesSplit[2])
                cv[fCount][int(linesSplit[0])]=float(linesSplit[3])
                ex[fCount][int(linesSplit[0])]=float(linesSplit[4])
                ey[fCount][int(linesSplit[0])]=float(linesSplit[5])
                ez[fCount][int(linesSplit[0])]=float(linesSplit[6])
            else:
                pc[fCount][int(linesSplit[1])]=float(linesSplit[3])
                pcv[fCount][int(linesSplit[1])]=float(linesSplit[4])
                pex[fCount][int(linesSplit[1])]=float(linesSplit[5])
                pey[fCount][int(linesSplit[1])]=float(linesSplit[6])
                pez[fCount][int(linesSplit[1])]=float(linesSplit[7])
        
    position=[x,y,z]
    velocity=[vx,vy,vz]
    chargeQV=[c,cv]
    electricField=[ex,ey,ez]
    platesEQV=[pex,pey,pez,pc,pcv]
    
    infoDict={"position":position,"velocity":velocity,"chargeQV":chargeQV,"electricField":electricField,"platesEQV":platesEQV}
    return infoDict

In [ ]:
#----------------------------------------------------------------------------------------------------------
def chargeDen(r,param):
    """Functions that determine the charge density at a point due to all the atoms in crystal or slab"""
    r=num.array(r)
    if param=="Z":
        fe=2.336509
        re=2.771916
        lam=0.826967
        beta=3.775974
        expArg=beta*(1-r/float(re))
        denoArg=1+(r/float(re)-lam)**20
        fr=fe*num.divide(num.exp(expArg),(denoArg))
    elif param=="D":
        rho=pan.read_csv("Pt_u3.rho.dat",sep="\t",names=["r","rho"])
        RhoF=pan.DataFrame(rho)
        fr=sci.interp(r,RhoF.r,RhoF.rho)
    else:
        print("Invalid entry: Chooose Daw(D) or Zhou(Z)")
    
    return fr


def dist(R,xf,yf,zf):
    r=num.ones(R.size).reshape(R.shape)
    r[0,:]=xf*r[0,:]
    r[1,:]=yf*r[1,:]
    r[2,:]=zf*r[2,:]
    diff=R-r
    diff2=diff**2
    sumdiff2=num.sum(diff2,axis=0)
    
    return num.sqrt(sumdiff2)

def partialCharge(q):
    return 1-(q/float(10))

def totalChargeDen(partial,chargeden):
    return(num.dot(partial,chargeden))

In [2]:
#-------------------------------------------------------------------------------------------------------------


def Layers(ZPosition,atomNumber):
    """
    Function that determines different layers in a crystal

    [layer,a]= Layers(ZPosition,atomNumber)
 
      Input:
     ========
 
         ZPosition: Z Coordinates of lattice for layer determination
         
         atomNumber: total Number of atoms in crystal
         
         
      Output:
     =========
         list [layer,a]; layer has index for atoms in each layers and "a" has the z-coordinates for each layers
         
         
"""
    a=num.sort(list(set(ZPosition[0,0:atomNumber-1])))
    layer=[]
    for var in a:
        layer.append(num.where(ZPosition[0]==var))
    
    return [layer,a]



#----------------------------------------------------------------------------------------------------------------


def partialChargeDist(partialCharge,layersIndex,ZpositionOfLayers,outPath):
    crange=num.linspace(0,1,num.ceil(len(ZpositionOfLayers)**.33))
    colors=[(cr,cb,cg) for cr in crange for cb in crange for cg in crange]
    fig = lab.figure(1)
    ax = fig.add_subplot(111)
    for counter in range(len(ZpositionOfLayers)):
        lab.plot(num.sum(partialCharge[:,layersIndex[counter][0]],axis=1)\
                 /len(layersIndex[counter][0]),colors[counter],label=positionLayers[counter])
    
    handles, labels = ax.get_legend_handles_labels()
    lab.xlabel("frames")
    lab.ylabel("average charge per site")
    lgd=ax.legend(bbox_to_anchor=(1.5,1),loc="upper right")
    lab.savefig(outPath,bbox_extra_artists=(lgd,), bbox_inches='tight')
    lab.show()
    
#--------------------------------------------------------------------------------------------------------------  

In [1]:
def LayerDipole(dumpFile,frames,atomNumber,atomPlate,UsedFrame,K,E,plotBool):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        pos=dump["position"]
        charge=dump["chargeQV"]
        layersInfo=Layers(pos[2],atomNumber)
        layer=layersInfo[0]
        a=layersInfo[1]
        averageChargeLayers=[]
        aveZpos=[]
    
    
        for counter in range(len(a)):
            averageChargeLayers.append(num.sum(charge[0][UsedFrame,layer[counter][0]])/float(len(layer[counter][0])))
            aveZpos.append(num.sum(pos[2][UsedFrame,layer[counter][0]])/float(len(layer[counter][0])))
        diff=[]
        precharge=0
        prez=0
        for counter in range(len(a)):
            diff.append((averageChargeLayers[counter]-precharge)*(aveZpos[counter]-prez))
            precharge=averageChargeLayers[counter]
            prez=aveZpos[counter]
    
         
        if plotBool==True:
            lab.plot(range(1,len(a)),diff[1:],'o-')
            lab.xlabel("Layers")
            lab.ylabel("LayerDipole")
            lab.title("E = "+str(E)+" || K = "+str(K))
            lab.grid()
            lab.show()
        dipole=num.sum(diff[1:])/float(len(diff)-1)
        
        return dipole
    
        
    except:
        return 99999
    
    
    

In [2]:
def SlabDipole(dumpFile,frames,atomNumber,atomPlate,UsedFrame):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        pos=dump["position"]
        charge=dump["chargeQV"]
        z=pos[2][UsedFrame,:]
        c=charge[0][UsedFrame,:]
        dipole=num.sum(z*c)
        return dipole
    
        
    except:
        return 99999
    

In [3]:
def LayerCharge(dumpFile,frames,atomNumber,atomPlate,K,E,plotBool):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        pos=dump["position"]
        charge=dump["chargeQV"]
        layersInfo=Layers(pos[2],atomNumber)
        layer=layersInfo[0]
        a=layersInfo[1]
        averageChargeLayers=[]
        
        
        
        if plotBool==True:
            colors=['b','b--', 'g--','g','r--','r','c--','c','m--','m','y--','y'\
                , 'k--','k','b-.','g-.','r-.','c-.','m-.','y-.','k-.']
            fig = plt.figure(1)
            ax = fig.add_subplot(111)
            print(0)
            for counter in range(len(a)):
                averageChargeLayers.append(num.sum(charge[0][:,layer[counter][0]],axis=1)/float(len(layer[counter][0])))
                lab.plot(averageChargeLayers[counter],colors[counter],label=a[counter])
            
            handles, labels = ax.get_legend_handles_labels()
            lgd=lab.legend(bbox_to_anchor=(1.5,1),loc="upper right")
            lab.xlabel("Frames")
            lab.ylabel("AverageCharge")
            lab.title("K = "+str(K)+"|| E="+str(E))
            lab.show()
            
    except:
        print("Corrupt file.")
    

In [4]:
def LayerPos(dumpFile,frames,atomNumber,atomPlate,K,E,plotBool):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        pos=dump["position"]
        layersInfo=Layers(pos[2],atomNumber)
        layer=layersInfo[0]
        a=layersInfo[1]
        averagePos=[]
        
        
        
        if plotBool==True:
            colors=['b','b--', 'g--','g','r--','r','c--','c','m--','m','y--','y'\
                , 'k--','k','b-.','g-.','r-.','c-.','m-.','y-.','k-.']
            fig = plt.figure(1)
            ax = fig.add_subplot(111)
            print(0)
            for counter in range(len(a)):
                averagePos.append(num.sum(pos[2][:,layer[counter][0]],axis=1)/float(len(layer[counter][0])))
                lab.plot(averagePos[counter],colors[counter],label=a[counter])
            
            handles, labels = ax.get_legend_handles_labels()
            lgd=lab.legend(bbox_to_anchor=(1.5,1),loc="upper right")
            lab.xlabel("Frames")
            lab.ylabel("AverageCharge")
            lab.title("K = "+str(K)+"|| E="+str(E))
            lab.show()
            
    except:
        print("Corrupt file.")

In [6]:
def DipoleStatFile(statFile):
    try:
        data=pan.DataFrame(pan.read_csv(statFile,sep="\t",header=1))
        p=num.sum(data.iloc[:,[9]][-10:])/10
        return p[0]
    except:
        print("Error in ::",statFile)
        

In [7]:
def SlopePE(alpha,atomNumber,units):
    if units=="AMU":
        return alpha*0.14818474*1.1126e-30*atomNumber
    elif units=="A^3":
        return alpha*1.1126e-30*atomNumber
    else:
        print("Error in units. Units must be \"AMU\"or \"A^3\" ")

In [ ]:
def ChargeDensityDist(posX,posY,posZ,pCharge,xBounds,yBounds,zValues,zLayers,frameUsed,paramUsed,outPath):
    xx=num.linspace(xBounds[0],xBounds[1],10)
    yy=num.linspace(yBounds[0],yBounds[1],10)
    xycord=[[x0, y0] for x0 in xx for y0 in yy]
    frameForAveraging=frameUsed
    AverageDensity=[]
    for framing in frameForAveraging:
        R=num.array([posX[framing,:-1],posY[framing,:-1],posZ[framing,:-1]])
        allChargeDist=[]
        zz=num.linspace(-zValues,zValues,1000)
        for xy in xycord:
            chargedist=[]
            for var in zz:
                chargedist.append(totalChargeDen(partialCharge(pCharge[framing,:]),\
                                                 chargeDen(dist(R,xy[0],xy[1],var),paramUsed)))
            allChargeDist.append(chargedist)
        allChargeDistMat=num.matrix(allChargeDist)
        averageChargeDistMat=num.sum(allChargeDist,axis=0)/float(len(xycord))
        AverageDensity.append(averageChargeDistMat)
    FinalAverageDensity=num.sum(num.matrix(AverageDensity),axis=0)/float(len(frameForAveraging))
    
    
    
    chargeFig=lab.figure(1)
    ax=chargeFig.add_subplot(111)
    for points in zLayers:
        ax.axvline(points,linestyle="--",color="red")
    avePos=num.sum(posZ[frameUsed],axis=0)/float(len(frameUsed))
    for var in avePos[:-1]:
        lab.axvline(var,linestyle="-.",color="blue")

    ax.axhline(0,linestyle="--",color="red")
    ax.plot(zz,averageChargeDistMat,label=str(framing))
    lgd=ax.legend(bbox_to_anchor=(1.5,1),loc="upper right")
    ax.grid(True)
    lab.xlabel("Z")
    lab.ylabel("charge_density")
    lab.savefig(outPath,bbox_extra_artists=(lgd,), bbox_inches='tight')
    lab.show()
    return [FinalAverageDensity,zz]
#-----------------------------------------------------------------------------------------------------------------


def diffDensity(refDensity,pertDensity,zCoordinate,zPosLayers,outPath):
    difference=pertDensity-refDensity
    diffDenFig=lab.figure(1)
    ax=diffDenFig.add_subplot(111)
    ax.plot(zCoordinate,num.transpose(difference))
    for points in zPosLayers:
        ax.axvline(points,linestyle="--",color="red")
    ax.axhline(0,linestyle="--",color="red")
    lab.xlabel("z")
    lab.ylabel(r'$\rho-\rho(0)$')
    lab.title(r'$\Delta\rho$---1Run')
    lab.savefig(outPath)
    lab.show()
    
#-----------------------------------------------------------------------------------------------------------------  

def platesEfield(pez,plateAtoms,outPath,frames):
    atomsPerPlate=plateAtoms/2
    lab.plot(num.sum(pez[:frames,0:int(atomsPerPlate)],axis=1)/atomsPerPlate,label="first")
    lab.plot(num.sum(pez[:frames,int(atomsPerPlate):plateAtoms],axis=1)/atomsPerPlate,label="second")
    lgd=lab.legend(bbox_to_anchor=(1.5,1),loc="upper right")
    lab.xlabel("frames")
    lab.ylabel("Ez per site")
    lab.title("ElectricField in each layers")
    lab.savefig(outPath)
    lab.show()
#----------------------------------------------------------------------------------------------------------------

def RefPertDensityAnalysis(ref,pert,frames,atomNumber,atomPlate,param,frameUsed,xRange,yRange,zlimit,outPathPlots):
    
    
    print("Extracting Info from reference Dump file")
    dump0=DumpExtractor(ref,frames,atomNumber,atomPlate)
    pos0=dump0["position"]
    vel0=dump0["velocity"]
    charge0=dump0["chargeQV"]
    plateInfo0=dump0["platesEQV"]
    layersInfo0=Layers(pos0[2],atomNumber)
    
    print("Electric Field on each plates:")
    platesEfield(plateInfo0[0],atomPlate,outPathPlots+"EfieldPlateE0.eps",frames)
    
    
    print("ChargeDensity distribution of the reference system")
    avDen0=ChargeDensityDist(pos0[0],pos0[1],pos0[2],charge0[0][:,:-1],xRange,yRange\
                         ,zlimit,layersInfo0[1],frameUsed\
                        ,param,outPathPlots+"ChargeDenstityDistE0.eps")
   
    partialChargeDist(charge0[0],layersInfo0[0],layersInfo0[1],outPathPlots\
                  +"partialChargeE0.eps")

    
    
    
    print("Extracting Info from pertubred Dump file")
    dump1=DumpExtractor(pert,frames,atomNumber,atomPlate)
    pos1=dump1["position"]
    vel1=dump1["velocity"]
    charge1=dump1["chargeQV"]
    plateInfo1=dump1["platesEQV"]
    layersInfo1=Layers(pos1[2],atomNumber)
    
    print("Electric Field on each plates perturbed system:")
    platesEfield(plateInfo1[0],atomPlate,outPathPlots+"EfieldPlate1VA.eps",frames)
    
    
    
    print("ChargeDensity distribution of the perturbed system")
    avDen1=ChargeDensityDist(pos1[0],pos1[1],pos1[2],charge1[0][:,:-1],xRange,yRange\
                         ,zlimit,layersInfo1[1],frameUsed\
                        ,param,outPathPlots+"ChargeDenstityDist1VA.eps")
    partialChargeDist(charge1[0],layersInfo1[0],layersInfo1[1],outPathPlots\
                  +"partialCharge1VA.eps")
    
    print("Difference in denstiy profile of pertubed system and reference system")
    diffDensity(avDen0[0],avDen1[0],avDen0[1],layersInfo0[1],outPathPlots+"DiffDensity1VA-E0.eps")
    
    
    
    
#-----------------------------------------------------------------------------------------------------------------

def DumpMerger(file1info,file2info):
    info1=DumpExtractor(file1info[0],file1info[1],file1info[2],file1info[3])
    info2=DumpExtractor(file2info[0],file2info[1],file2info[2],file2info[3])
    
    finalpositionX=num.concatenate((info1["position"][0],info2["position"][0]),axis=0)
    finalpositionY=num.concatenate((info1["position"][1],info2["position"][1]),axis=0)
    finalpositionZ=num.concatenate((info1["position"][2],info2["position"][2]),axis=0)
    finalposition=[finalpositionX,finalpositionY,finalpositionZ]
    
    
    finalVelocityX=num.concatenate((info1["velocity"][0],info2["velocity"][0]),axis=0)
    finalVelocityY=num.concatenate((info1["velocity"][1],info2["velocity"][1]),axis=0)
    finalVelocityZ=num.concatenate((info1["velocity"][2],info2["velocity"][2]),axis=0)
    finalVelocity=[finalVelocityX,finalVelocityY,finalVelocityZ]
    
    finalChargeQ=num.concatenate((info1["chargeQV"][0],info2["chargeQV"][0]),axis=0)
    finalChargeV=num.concatenate((info1["chargeQV"][1],info2["chargeQV"][1]),axis=0)
    finalChargeQV=[finalChargeQ,finalChargeV]
    
    finalElectricFieldX=num.concatenate((info1["electricField"][0],info2["electricField"][0]),axis=0)
    finalElectricFieldY=num.concatenate((info1["electricField"][1],info2["electricField"][1]),axis=0)
    finalElectricFieldZ=num.concatenate((info1["electricField"][2],info2["electricField"][2]),axis=0)
    finalElectricField=[finalElectricFieldX,finalElectricFieldY,finalElectricFieldZ]
    
    finalPlatesEx=num.concatenate((info1["platesEQV"][0],info1["platesEQV"][0]),axis=0)
    finalPlatesEy=num.concatenate((info1["platesEQV"][1],info1["platesEQV"][1]),axis=0)
    finalPlatesEz=num.concatenate((info1["platesEQV"][2],info1["platesEQV"][2]),axis=0)
    finalPlatesQ=num.concatenate((info1["platesEQV"][3],info1["platesEQV"][3]),axis=0)
    finalPlatesV=num.concatenate((info1["platesEQV"][4],info1["platesEQV"][4]),axis=0)
    
    finalPlatesEQV=[finalPlatesEx,finalPlatesEy,finalPlatesEz,finalPlatesQ,finalPlatesV]
    
    infoDict={"position":finalposition,"velocity":FinalVelocity,"chargeQV":finalChargeQV,\
              "electricField":finalElectricField,"platesEQV":FinalPlatesEQV}
    return infoDict
    
#---------------------------------------------------------------------------------------------------------------    
   

def mergedDumpFiles(path,files,outName):
    mergedFile=open(path+outName,"a+")
    mergedFile.write(" ")
    for f in files:
        newfile=open(path+f)
        newfile_content=newfile.read()
        newfile.close()
        mergedFile_content=mergedFile.read()
        mergedFile.write(mergedFile_content+newfile_content)
    mergedFile.close()
        